# Nibiru Staking Queries and Transactions
Examples of querying validators, delegating and unbonding.

### 1. Install Nibiru package

In [ ]:
!pip install nibiru

### 2. Create chain client and run queries
Using testnet-3 (aka itn-3) endpoint run queries.

In [ ]:
import json
from nibiru import Network, ChainClient, Msg, Coin, TxConfig

client = ChainClient(network=Network.testnet(3))

In [ ]:
# Shortcut to print responses
def print_json(obj):
    print(json.dumps(obj, indent=2))

In [ ]:
# Query validators
validators = client.query.staking.validators()["validators"]
print_json(validators)

first_validator = validators[0]

In [ ]:
# Query delegations to the first validator
delegations = client.query.staking.delegations_to(first_validator["operator_address"])["delegation_responses"]
print_json(delegations)

### 3. Authenticate to run transactions
Assuming you have your wallet mnemonic key (or private key) and some test tokens from the faucet.
See: https://app.nibiru.fi/faucet

In [ ]:
mnemonic = "put your mnemonic here..."

# Let's set high gas multiplier to avoid out of gas errors. Could be customized per tx.
tx_config = TxConfig(gas_multiplier=5)
client = ChainClient(Network.testnet(3), tx_config=tx_config)
client.authenticate(mnemonic=mnemonic)

# Balances check
balances = client.query.get_bank_balances(client.address)
print_json(balances["balances"])

### 4. Run Staking Transactions
Delegate to a validator:

In [ ]:
output = client.tx.execute_msgs(
    Msg.staking.delegate(
        validator_address=first_validator["operator_address"],
        amount=10, # amount in unibi
    ),
    wait_for_tx_resp=True,  # wait for block and get tx response
)
# Expected tx response code 0
print(f"tx response Code: {output['code']}")
if output["code"] == 0:
    delegations = client.query.staking.delegations(delegator_addr=client.address)["delegation_responses"]
    print_json(delegations)

Undelegate (unbond) tokens from validator:

In [ ]:
output = client.tx.execute_msgs(
    Msg.staking.undelegate(
        validator_address=first_validator["operator_address"],
        amount=10, # amount in unibi
    ),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    unbondings = client.query.staking.unbonding_delegations(delegator_addr=client.address)["unbonding_responses"]
    print_json(unbondings)